<a href="https://colab.research.google.com/github/tensorflow/lingvo/blob/master/codelabs/introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Lingvo

This codelab will guide you through the implementation of a **sequence-to-sequence** model using [**Lingvo**](https://github.com/tensorflow/lingvo).

**Sequence-to-sequence** models map input sequences of arbitrary length to
output sequences of arbitrary length. Example uses of sequence-to-sequence
models include machine translation, which maps a sequence of words from one
language into a sequence of words in another language with the same meaning;
speech recognition, which maps a sequence of acoustic features into a sequence
of words; and text summarization, which
maps a sequence of words into a shorter sequence which conveys the same meaning.

In this codelab, you will create a model which restores punctuation and
capitalization to text which has been lowercased and stripped of punctuation.
For example, given the following text:

> she asked do you know the way to san jose

The model will output the following properly-punctuated-and-capitalized text:

> She asked, "Do you know the way to San Jose"?

We will train an RNMT+ model based off of ["The Best of Both Worlds: Combining Recent Advances in Neural Machine Translation. (Chen et al., 2018)"](https://arxiv.org/abs/1804.09849).

## Table of Contents

In Colab, click `[View]->[Table of contents]` on the menu bar.

## Prerequisites

The main goal of this codelab is to teach you how to define and train sequence-to-sequence models in Lingvo. We do not aim to teach either Python or Tensorflow, and no sophisticated Python or Tensorflow programming will be required. However, the following will be helpful in understanding this codelab:

-   Familiarity with high-level machine learning primitives, in particular,
    recurrent neural networks, LSTMs, and attention.
-   Comfort reading and writing simple Python code. In particular, you should
    know how to define simple classes and how inheritance works.
-   Basic knowledge of the Tensorflow training workflow.  If you have trained
    simple Tensorflow models before (e.g., via another codelab), you should know
    enough for this codelab.

### Resources

- [Introduction to RNNs and LSTMs](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)
- [Official Tensorflow Tutorials](https://www.tensorflow.org/tutorials/)
- For more advanced topics or to get a deeper understanding of Lingvo beyond this codelab, see the [paper](https://arxiv.org/abs/1902.08295).

## Learning Objectives

This codelab will teach you the following:

-   How to generate input data for training a sequence-to-sequence model in Lingvo.
-   How models are specified and configured in Lingvo, by adapting a pre-existing model architecture for machine translation.
-   How to use the trained model for inference.

This codelab does not:

-   Teach you how to design a model for solving specific tasks.
-   Provide a state-of-the-art model for the punctuator task.

## Environment Setup

To start with, we need to connect this Colab notebook with Lingvo.

```shell
mkdir -p /tmp/lingvo_codelab && cd /tmp/lingvo_codelab
pip3 install lingvo
python3 -m lingvo.ipython_kernel
```

Finally, on the top right hand side of this Colab notebook, open the dropdown beside "CONNECT" and select "Connect to local runtime...", enter `http://localhost:8888` and press CONNECT.

You should now see the words "CONNECTED" and be able to execute the following cell.

In [ ]:
import lingvo

## Input Pipeline

In order to train a sequence-to-sequence model, we need a set of pairs of source
and target sequences. For this codelab, our source sequences will be
text which has been lowercased and had its punctuation removed, and the target
sequences will be the original sentences, with their original casing and
punctuation.

Since neural networks require numeric inputs, we will also need a tokenization scheme mapping the sequence of characters to a sequence of numbers. In this codelab, we will use a pre-trained word-piece model.

### Download Raw Input

We will use the [Brown Corpus](http://www.nltk.org/nltk_data) as the source of our training data. Run the following cell to download and preprocess the dataset. The script will generate `train.txt` and `test.txt` containing the training and test data at an 80:20 split with individual sentences on each line.

In [ ]:
!python3 -m lingvo.tasks.punctuator.tools.download_brown_corpus --outdir=/tmp/punctuator_data
!curl -O https://raw.githubusercontent.com/tensorflow/lingvo/master/lingvo/tasks/punctuator/params/brown_corpus_wpm.16000.vocab

### Define an Input Generator

In order to train a model, we need an input generator that provides structured mini-batches of source-target pairs. The input generator handles all the processing necessary to generate numeric data that can be fed to the model. This includes:

- reading examples from the data source in random order, where the data source may be split across multiple files;
- processing the data -- for our task this involves generating a "source" sentence by converting all characters to lower-case and removing punctuation, and then tokenizing both the source and target sequences into integer tokens; and
- batching together examples by padding them to the same length. Multiple buckets of different lengths may be used to avoid inefficiency from padding a short input to a very long length.

Fortunately, the majority of this is handled in the background by Lingvo. We only need to specify how the data should be processed.

Input generators are subclasses of *BaseInputGenerator* found in [lingvo/core/base_input_generator.py](https://github.com/tensorflow/lingvo/blob/master/lingvo/core/base_input_generator.py). For this codelab, we will create a [*tf.data*](https://www.tensorflow.org/guide/data)-based input generator by defining a Lingvo [*DataSource*](https://github.com/tensorflow/lingvo/blob/master/lingvo/core/datasource.py).

To do so, we need to implement:
- a `Params` classmethod that returns a default Params object for configuring the input generator. Experiment configurations inside Lingvo are controlled using these Params objects.
- a `CreateDatasource` method that augments the `p.file_datasource` param to describe a full input pipeline.

Here is an example of the input generator for the Punctuator task, found in [lingvo/tasks/punctuator/input_generator.py](https://github.com/tensorflow/lingvo/blob/master/lingvo/tasks/punctuator/input_generator.py).

Run the cell below to write the file to disk.

In [ ]:
%%writefile input_generator.py
import string
import lingvo.compat as tf
from lingvo.core import base_input_generator
from lingvo.core import datasource
from lingvo.core import py_utils
from lingvo.core import tokenizers


class TextLines(datasource.TFDatasetSource):
  """Returns a tf.data.Dataset containing lines from a text file."""

  @classmethod
  def Params(cls):
    p = super().Params()
    p.Define('file_pattern', None, 'A file pattern string.')
    p.Define('shuffle_buffer_size', 10000,
             'Number of records buffered for random shuffling.')
    return p

  def GetDataset(self):
    p = self.params
    if not p.file_pattern:
      raise ValueError('A file pattern must be provided.')
    file_pattern_glob = py_utils.ShardedFilePatternToGlob(p.file_pattern)
    dataset = tf.data.Dataset.list_files(
        file_pattern_glob,
        shuffle=not self.cluster.require_sequential_input_order)
    dataset = tf.data.TextLineDataset(
        dataset,
        num_parallel_reads=(1 if self.cluster.in_unit_test else
                            tf.data.experimental.AUTOTUNE))

    if not self.cluster.require_sequential_input_order:
      dataset = dataset.shuffle(
          p.shuffle_buffer_size, reshuffle_each_iteration=True)
      dataset = dataset.repeat()
    return dataset


class PunctuatorInput(base_input_generator.BaseInputGenerator):
  """Reads text line by line and processes them for the punctuator task.

  Input batches are NestedMaps containing:
  
  - src.ids: int32 source word-piece ids of shape [batch, p.source_max_length].
  - src.paddings: float32 paddings of shape [batch, p.source_max_length] where
    paddings == 0.0 if the position is part of the input and 1.0 if the position
    is padding.
  - tgt.ids: int32 target word-piece ids of shape [batch, p.target_max_length].
  - tgt.labels = int32 target label word-piece ids of shape
    [batch, p.target_max_length]. The difference between labels and ids is that
    ids include the sos (start of sequence) token in the front and labels
    include the eos (end of sequence) token in the end.
  - tgt.paddings: float32 paddings of shape [batch, p.target_max_length].
  - tgt.weights: float32 weights of shape [batch, p.target_max_length]. Weights
    are generally 1.0 - paddings and are used for loss computation.
  - bucket_keys: int32 value used for bucketing. This is set automatically by
    TFDatasetBatchBySequenceLength, and is for debugging purposes.
  """

  @classmethod
  def Params(cls):
    """Defaults params for PunctuatorInput."""
    p = super().Params()
    p.file_datasource = TextLines.Params()
    p.Define('tokenizer', tokenizers.WpmTokenizer.Params(), 'Tokenizer params.')
    p.Define('source_max_length', None,
             'The maximum length of the source sequence.')
    p.Define('target_max_length', None,
             'The maximum length of the target sequence.')
    p.Define(
        'bucket_upper_bound', [], 'Bucketing scheme. Required to be'
        'a sorted list of integers. Examples that are longer than all bucket'
        'upper bounds are skipped.')
    p.Define(
        'bucket_batch_limit', [], 'Desired per-split batch size per bucket. '
        'Must be the same length as bucket_upper_bound.')
    return p

  def __init__(self, params):
    super().__init__(params)
    p = self.params
    self.CreateChild('tokenizer', p.tokenizer)

  def CreateDatasource(self):
    p = self.params
    ds = p.file_datasource
    ds = datasource.CustomTFDatasetTransform.Params().Set(sub=ds, fn='_Process')
    # See documentation for TFDatasetBatchBySequenceLength for the specifics of
    # how the bucketing process works.
    ds = datasource.TFDatasetBatchBySequenceLength.Params().Set(
        sub=ds,
        seqlen_fn='_GetSequenceLength',
        input_shape_fn='_InputShape',
        input_padding_fn='_InputPaddingValue',
        bucket_upper_bound=p.bucket_upper_bound,
        bucket_batch_limit=p.bucket_batch_limit)
    p.file_datasource = ds
    super().CreateDatasource()

  def _Process(self, dataset):
    """Processes the dataset containing individual lines."""
    return dataset.map(self._ProcessLine, **self._map_args)

  def _ProcessLine(self, line):
    """A single-text-line processor.

    Gets a string tensor representing a line of text that have been read from
    the input file, and splits it to graphemes (characters).
    We use original characters as the target labels, and the lowercased and
    punctuation-removed characters as the source labels.

    Args:
      line: a 1D string tensor.

    Returns:
      A NestedMap containing the processed example.
    """
    p = self.params
    # Tokenize the input into integer ids.
    # tgt_ids has the start-of-sentence token prepended, and tgt_labels has the
    # end-of-sentence token appended.
    tgt_ids, tgt_labels, tgt_paddings = self.tokenizer.StringsToIds(
        tf.convert_to_tensor([line]), p.target_max_length)
    # Because StringsToIds requires a vector but _ProcessLine is called for
    # individual lines, we need to manually remove the batch dimension.
    tgt_ids = tgt_ids[0]
    tgt_labels = tgt_labels[0]
    tgt_paddings = tgt_paddings[0]

    # This normalization function produces the "source" text from which the
    # Punctuator task is trained to reproduce the original "target" text.
    def Normalize(line):
      # Lowercase and remove punctuation.
      line = line.lower().translate(None, string.punctuation.encode('utf-8'))
      # Convert multiple consecutive spaces to a single one.
      line = b' '.join(line.split())
      return line

    normalized_line = tf.py_func(Normalize, [line], tf.string, stateful=False)
    _, src_labels, src_paddings = self.tokenizer.StringsToIds(
        tf.convert_to_tensor([normalized_line]), p.source_max_length)
    # Because StringsToIds requires a vector but _ProcessLine is called for
    # individual lines, we need to manually remove the batch dimension.
    src_labels = src_labels[0]
    src_paddings = src_paddings[0]
    # The model expects the source without a start-of-sentence token.
    src_ids = src_labels
    tgt_weights = 1.0 - tgt_paddings

    ret = py_utils.NestedMap()

    ret.src = py_utils.NestedMap()
    ret.src.ids = tf.cast(src_ids, dtype=tf.int32)
    ret.src.paddings = src_paddings

    ret.tgt = py_utils.NestedMap()
    ret.tgt.ids = tgt_ids
    ret.tgt.labels = tf.cast(tgt_labels, dtype=tf.int32)
    ret.tgt.weights = tgt_weights
    ret.tgt.paddings = tgt_paddings
    return ret

  def _GetSequenceLength(self, example):
    """Returns sequence length for the example NestedMap from the dataset.

    This function is used by the TFDatasetBatchBySequenceLength DataSource to
    obtain the key used for bucketing. Bucketing separates examples into
    groups before batching, such that each batch contains only examples within a
    certain length.

    Args:
      example: A NestedMap containing an input example. Tensors in the example
        do not have a leading batch dimension.

    Returns:
      An integer sequence length for the example.
    """
    return tf.cast(
        tf.round(
            tf.maximum(
                tf.reduce_sum(1.0 - example.src.paddings),
                tf.reduce_sum(1.0 - example.tgt.paddings))), tf.int32)

  def _InputShape(self, key):
    """Returns the final shape of the tensor corresponding to key as a tuple.

    The shape should not include a leading batch dimension.

    This function is used by the TFDatasetBatchBySequenceLength DataSource to
    specify the shape for each key in an example. Because sequence examples are
    of different lengths, they need to be padded to a common shape for batching.

    Args:
      key: The NestedMap key to return shape for.
    """
    p = self.params
    if key == 'bucket_keys':
      return ()
    if key.startswith('src.'):
      return [p.source_max_length]
    if key.startswith('tgt.'):
      return [p.target_max_length]
    raise ValueError('Unexpected key %s' % key)

  def _InputPaddingValue(self, key, tensorspec):
    """Returns a scalar value to pad the tensor corresponding to key with.

    This function is used by the TFDatasetBatchBySequenceLength DataSource to
    specify the value used for padding.

    Args:
      key: The NestedMap key to return padding value for.
      tensorspec: a tf.TensorSpec describing the tensor to be padded.
    """
    if key.endswith('_paddings'):
      return tf.ones([], dtype=tensorspec.dtype)
    else:
      return tf.zeros([], dtype=tensorspec.dtype)

## Model Definition

Next, we need to define the network structure for the task. The network is a nested structure of layers. Most classes in Lingvo are subclasses of *BaseLayer* found in [lingvo/core/base_layer.py](https://github.com/tensorflow/lingvo/blob/560f838bd576c7b911df379121eb58252b6ae326/lingvo/core/base_layer.py#L150) and inherit the following:

- a Params classmethod that returns a default [Params](https://github.com/tensorflow/lingvo/blob/3344e201719961183d88713784ccbae447f5c52a/lingvo/core/hyperparams.py#L151) object for configuring the class. In addition to hyperparameters, the Params object can also contain Params objects for configuring child layers. Some of the properties present in all Params objects include:
  - `cls`: the python class that the Params object is associated with. This can be used to construct an instance of the class;
  - `name`: the name of this layer;
  - `dtype`: the default dtype to use when creating variables.
- The `__init__` constructor. All child layers and variables should be created here.
- A `CreateVariable` method that is called to create variables.
- A `CreateChild` method that is called to create child layers.
- A `FProp` method that implements forward propagation through the layer.

As a reference, many examples of layers can be found in [lingvo/core/layers.py](https://github.com/tensorflow/lingvo/blob/master/lingvo/core/layers.py), [lingvo/core/attention.py](https://github.com/tensorflow/lingvo/blob/master/lingvo/core/attention.py), and [lingvo/core/rnn_layers.py](https://github.com/tensorflow/lingvo/blob/master/lingvo/core/rnn_layers.py).

&nbsp;

The root layer for the network should be a subclass of `BaseTask` found in [lingvo/core/base_model.py](https://github.com/tensorflow/lingvo/blob/918c584f057481717eff6e1e29ae028aeab3d165/lingvo/core/base_model.py#L79), and implements the following:

- A `ComputePredictions` method that takes the current variable values (`theta`) and `input_batch` and returns the network predictions.
- A `ComputeLoss` method that takes `theta`, `input_batch`, and the `predictions` returned from `ComputePredictions` and returns a dictionary of scalar metrics, one of which should be `loss`. These scalar metrics are exported to TensorBoard as summaries.
- An optional `Decode` method for creating a separate graph for decoding. For example, training and evaluation might use teacher forcing while decoding might not.
- An optional `Inference` method that returns a graph with feeds and fetches that can be used together with a saved checkpoint for inference. This differs from `Decode` in that it can be fed data directly instead of using data from the input generator.

&nbsp;

This codelab uses the existing networks from [lingvo/tasks/punctuator/model.py](https://github.com/tensorflow/lingvo/blob/master/lingvo/tasks/punctuator/model.py), which is derived from the networks in [lingvo/tasks/mt/model.py](https://github.com/tensorflow/lingvo/blob/master/lingvo/tasks/mt/model.py) with an added `Inference` method for the punctuator task. The actual logic lies mostly in [lingvo/tasks/mt/encoder.py](https://github.com/tensorflow/lingvo/blob/master/lingvo/tasks/mt/encoder.py) and [lingvo/tasks/mt/decoder.py](https://github.com/tensorflow/lingvo/blob/master/lingvo/tasks/mt/decoder.py).

## Model Configuration

After defining the input generator and the network, we need to create an model configuration with the specific hyperparameters to use for our model.

Since there is only a single task, we create a subclass of `SingleTaskModelParams` found in [lingvo/core/base_model_params.py](https://github.com/tensorflow/lingvo/blob/4747cf80a7e6cf58211aa899bae854820a3b42f6/lingvo/core/base_model_params.py#L47). It has the following structure:

- The `Train`/`Dev`/`Test` methods configure the input generator for the respective datasets.
- The `Task` method configures the network.

The following cell contains the configuration that will be used in this codelab. It can also be found in [lingvo/tasks/punctuator/params/codelab.py](https://github.com/tensorflow/lingvo/blob/master/lingvo/tasks/punctuator/params/codelab.py). The network configuration in the `Task` classmethod is delegated to [lingvo/tasks/mt/params/base_config.py](https://github.com/tensorflow/lingvo/blob/master/lingvo/tasks/mt/params/base_config.py).

Run the cell below to write the file to disk.

In [ ]:
%%writefile codelab.py
import input_generator
import os
from lingvo import model_registry
import lingvo.compat as tf
from lingvo.core import base_model_params
from lingvo.tasks.mt import base_config
from lingvo.tasks.punctuator import model


# This base class defines parameters for the input generator for a specific
# dataset. Specific network architectures will be implemented in subclasses.
class BrownCorpusWPM(base_model_params.SingleTaskModelParams):
  """Brown Corpus data with a Word-Piece Model tokenizer."""

  # Generated using
  # third_party/py/lingvo/tasks/punctuator/tools:download_brown_corpus.
  _DATADIR = '/tmp/punctuator_data'
  _VOCAB_FILE = tf.resource_loader.get_path_to_datafile(
      'brown_corpus_wpm.16000.vocab')
  # _VOCAB_SIZE needs to be a multiple of 16 because we use a sharded softmax
  # with 16 shards.
  _VOCAB_SIZE = 16000

  def Train(self):
    p = input_generator.PunctuatorInput.Params()
    p.file_datasource.file_pattern = os.path.join(self._DATADIR, 'train.txt')

    # The bucket upper bound specifies how to split the input into buckets. We
    # train on sequences up to maximum bucket size and discard longer examples.
    p.bucket_upper_bound = [10, 20, 30, 60, 120]

    # The bucket batch limit determines how many examples are there in each
    # batch during training. We reduce the batch size for the buckets that
    # have higher upper bound (batches that consist of longer sequences)
    # in order to prevent out of memory issues.
    # Note that this hyperparameter varies widely based on the model and
    # language. Larger models may warrant smaller batches in order to fit in
    # memory, for example; and ideographical languages like Chinese may benefit
    # from more buckets.
    p.bucket_batch_limit = [512, 256, 160, 80, 40]

    p.tokenizer.vocab_filepath = self._VOCAB_FILE
    p.tokenizer.vocab_size = self._VOCAB_SIZE
    p.tokenizer.pad_to_max_length = False

    # Set the tokenizer max length slightly longer than the largest bucket to
    # discard examples that are longer than we allow.
    p.source_max_length = p.bucket_upper_bound[-1] + 2
    p.target_max_length = p.bucket_upper_bound[-1] + 2
    return p

  # There is also a Dev method for dev set params, but we don't have a dev set.
  def Test(self):
    p = input_generator.PunctuatorInput.Params()
    p.file_datasource.file_pattern = os.path.join(self._DATADIR, 'test.txt')

    p.bucket_upper_bound = [10, 20, 30, 60, 120, 200]
    p.bucket_batch_limit = [16] * 4 + [4] * 2

    p.tokenizer.vocab_filepath = self._VOCAB_FILE
    p.tokenizer.vocab_size = self._VOCAB_SIZE
    p.tokenizer.pad_to_max_length = False

    p.source_max_length = p.bucket_upper_bound[-1] + 2
    p.target_max_length = p.bucket_upper_bound[-1] + 2
    return p


# This decorator registers the model in the Lingvo model registry.
# This file is third_party/py/lingvo/tasks/punctuator/params/codelab.py,
# so the model will be registered as punctuator.codelab.RNMTModel.
@model_registry.RegisterSingleTaskModel
class RNMTModel(BrownCorpusWPM):
  """RNMT+ Model."""

  def Task(self):
    p = base_config.SetupRNMTParams(
        model.RNMTModel.Params(),
        name='punctuator_rnmt',
        vocab_size=self._VOCAB_SIZE,
        embedding_dim=1024,
        hidden_dim=1024,
        num_heads=4,
        num_encoder_layers=6,
        num_decoder_layers=8,
        learning_rate=1e-4,
        l2_regularizer_weight=1e-5,
        lr_warmup_steps=500,
        lr_decay_start=400000,
        lr_decay_end=1200000,
        lr_min=0.5,
        ls_uncertainty=0.1,
        atten_dropout_prob=0.3,
        residual_dropout_prob=0.3,
        adam_beta2=0.98,
        adam_epsilon=1e-6,
    )
    p.eval.samples_per_summary = 2466
    return p

## Model Training

The following cell trains the model. Note that this will require approximately 2.5GB of space in `logdir`.

In [ ]:
# Start tensorboard (access at http://localhost:6006)
import os
os.system('lsof -t -i:6006 || tensorboard --logdir=/tmp/punctuator &')

!python3 -m lingvo.trainer --model=codelab.RNMTModel --mode=sync --logdir=/tmp/punctuator --saver_max_to_keep=2 --noenable_asserts --run_locally=gpu

The following cell evaluates the model. In Lingvo, evaluation is meant to be run alongside training as a separate process that periodically looks for the latest checkpoint and evaluates it. There is only one process in Colab so running this cell will evaluate the current checkpoint then it will block indefinitely waiting for the next checkpoint.

In [ ]:
!python3 -m lingvo.trainer --model=codelab.RNMTModel --job=evaler_test --logdir=/tmp/punctuator --run_locally=cpu

There is also a Decoder job that can be run the same way. The difference between the Evaler and Decoder varies by model.

## Model Inference

After the model has been trained for around 10-20k steps (a few hours on GPU), its inference graph can be used to interact with the model using arbitrary inputs.

In [ ]:
import string

from lingvo import compat as tf
from lingvo import model_imports
from lingvo import model_registry
from lingvo.core import inference_graph_exporter
from lingvo.core import predictor
from lingvo.core.ops.hyps_pb2 import Hypothesis

tf.flags.FLAGS.mark_as_parsed()


src = "she asked do you know the way to san jose" #@param {type:'string'}
src = src.lower().translate(str.maketrans('', '', string.punctuation))
print(src)

checkpoint = tf.train.latest_checkpoint('/tmp/punctuator/train')
print('Using checkpoint %s' % checkpoint)

# Run inference
params = model_registry.GetParams('codelab.RNMTModel', 'Test')
inference_graph = inference_graph_exporter.InferenceGraphExporter.Export(params)
pred = predictor.Predictor(inference_graph, 
                           checkpoint=checkpoint, 
                           device_type='cpu')
src_ids, decoded, scores, hyps = pred.Run(
    ['src_ids', 'topk_decoded', 'topk_scores', 'topk_hyps'], src_strings=[src])
# print(src_ids[0])
for text, score in zip(decoded[0].tolist(), scores[0].tolist()):
  print("%.5f: %s" % (score, text))
# for i, hyp in enumerate(hyps[0]):
#   print("=======hyp %d=======" % i)
#   print(Hypothesis().FromString(hyp))

Footnote: One might wonder why our result places the question mark outside the quotation. This happens because the Brown corpus follows a 1959 US Patent Office precedure of transliterating texts into punch cards, where the closing question mark is always punched before the punctuation, so that the punctuation mark occurs at the end of the sentence. See [this link](http://clu.uni.no/icame/manuals/BROWN/INDEX.HTM) for more details. Our result is just following this pattern in our training data.

## Additional Resources

For more advanced topics or to get a deeper understanding of Lingvo beyond this codelab, see the [paper](https://arxiv.org/abs/1902.08295).